In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149827 sha256=671b93fcb43eebe54285ba74c47c550f7c9d8bc35e1a267ea4ed79d7b8e354a2
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import pandas as pd
import Levenshtein
import numpy as np
import math

## Put IDs of papers in main csv

In [ ]:
df_main = pd.read_csv('/content/drive/MyDrive/Citation/filtered_Aminer_dataset.csv')

#IDs poolA_s2url
df_a = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolA_derworks.csv')
df_b = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolB_derworks.csv')
df_c = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolC_derworks.csv')

In [ ]:
df_main.columns

Index(['title', 'authors', 'n_citation', 'doi', 'url', 'pdf', 'conf', 'fos',
       'year', 'pool', 'topic', 'labels'],
      dtype='object')

In [ ]:
df_a.columns

Index(['bName', 'dNames', 'dIDs'], dtype='object')

In [ ]:
def clean(name):
  to_remove = ['-',',',':',';','.','?','(',')','/']
  name = name.lower()
  name = name.strip()

  for char in to_remove:
    name = name.replace(char,'')
  name = name.replace('&','and')

  return(name)

def check_intersection2(t1,t2,thresh=0.8):
  t1 = set(t1.split(' '))
  t2 = set(t2.split(' '))
  # print(t1,t2)
  intersect = len(t1.intersection(t2))
  if(intersect/len(t2))>=thresh and (intersect/len(t1))>=thresh:
    return(1)
  return(0)

def check_nameInList(name,nameList):
  name = clean(name)
  nameList = [clean(x) for x in nameList]
  for i in range(len(nameList)):
    n = nameList[i]
    if(check_intersection2(name,n)):
      return(n,i)
  return('',-1)

In [ ]:
def getIDs(df_main, df_a,df_b,df_c):
  name_main = list(df_main['title'])
  pools = list(df_main['pool'])
  
  name_a = list(df_a['bname'])
  name_b = list(df_b['bname'])
  name_c = list(df_c['bname'])

  # id_a = list(df_a['s2_url'])
  # id_a = [x.split('/')[-1].split(':')[-1] for x in id_a]

  # id_b = list(df_b['s2_url'])
  # id_b = [x.split('/')[-1].split(':')[-1] for x in id_b]

  # id_c = list(df_c['s2_url'])
  # id_c = [x.split('/')[-1].split(':')[-1] for x in id_c]

  # IDs = []
  name_match = []
  for i in range(len(name_main)):
    n = name_main[i]
    p = pools[i]
    if(p=='A'):
      n2, ind = check_nameInList(n,name_a)
      if(ind!=-1):
        IDs.append(id_a[ind])
        name_match.append(n2)
      else:
        IDs.append('')
        name_match.append('')

    elif(p=='B'):
      n2, ind = check_nameInList(n,name_b)
      if(ind!=-1):
        IDs.append(id_b[ind])
        name_match.append(n2)
      else:
        IDs.append('')
        name_match.append('')

    elif(p=='C'):
      n2, ind = check_nameInList(n,name_c)
      if(ind!=-1):
        IDs.append(id_c[ind])
        name_match.append(n2)
      else:
        IDs.append('')
        name_match.append('')

  return(IDs,name_match, name_main)

# print(names[0])

In [ ]:
ids,name_match,name_main = getIDs(df_main, df_a,df_b,df_c)

In [ ]:
ids.count('')

2182

In [ ]:
len(df_main)

5693

In [ ]:
'''
80% 2202
90% 2378
'''

'\n80% 2202\n90% 2378\n'

In [ ]:
d = {'s2orc':name_match,'name_main':name_main,'ids':ids}
df = pd.DataFrame(d)
df.head()
df.to_csv('/content/drive/MyDrive/Citation/check_name_match.csv',index=False)

In [ ]:
def check_lev(n,namelist):
  min_d = 5
  # min_name = ''
  ind=-1
  for i in range(len(namelist)):
    n1 = namelist[i]
    d = Levenshtein.distance(n,n1)
    if(d<min_d):
      min_d = d
      # min_name = n1
      ind=i
  return(ind)

def checkIn(n,namelist):
  for i in range(len(namelist)):
    if(n==namelist[i]):
      return(i)

  return(-1)

def getIDs(df_main, df_a,df_b,df_c):
  name_main = list(df_main['title'])
  pools = list(df_main['pool'])
  
  name_a = list(df_a['bName'])
  name_b = list(df_b['bName'])
  name_c = list(df_c['bName'])

  name_a = [clean(x) for x in name_a]
  name_b = [clean(x) for x in name_b]
  name_c = [clean(x) for x in name_c]
  

  # id_a = list(df_a['s2_url'])
  # id_a = [x.split('/')[-1].split(':')[-1] for x in id_a]

  # id_b = list(df_b['s2_url'])
  # id_b = [x.split('/')[-1].split(':')[-1] for x in id_b]

  # id_c = list(df_c['s2_url'])
  # id_c = [x.split('/')[-1].split(':')[-1] for x in id_c]

  # IDs = []
  name_match = []
  for i in range(len(name_main)):
    n = clean(name_main[i])
    p = pools[i]
    
    def check_pool(n,namelist,ret_name_match):
        ind = checkIn(n,namelist)
        if (ind!=-1):
          # ret_IDs.append(idlist[ind])
          ret_name_match.append(namelist[ind])
        else:
          ind = check_lev(n,namelist)
          if (ind!=-1):
            # ret_IDs.append(idlist[ind])
            ret_name_match.append(namelist[ind])
          else:
            # ret_IDs.append('')
            ret_name_match.append('')

    if p=='A':
      check_pool(n,name_a,name_match)

    elif p=='B':
      check_pool(n,name_b,name_match)
    
    elif p=='C':
      check_pool(n,name_c,name_match)

  return(name_match, name_main)

In [ ]:
name_match,name_main = getIDs(df_main, df_a,df_b,df_c)

In [ ]:
name_match.count('')

1995

In [ ]:
d = {'s2orc':name_match,'name_main':name_main}
df = pd.DataFrame(d)
df.head()
df.to_csv('/content/drive/MyDrive/Citation/check_name_match_lev_bname.csv',index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Citation/check_name_match_lev_bname.csv')

In [ ]:
dict_id = dict()
dict_bibname = dict()

for index,row in df.iterrows():
  if(row['s2orc']!=''):
    # dict_id[row['name_main']] = row['ids']
    dict_bibname[row['name_main']] = row['s2orc']

ids = []
bibnames = []
for index,row in df_main.iterrows():
  # ids.append(dict_id.get(row['title'],''))
  bibnames.append(dict_bibname.get(row['title'],''))

In [ ]:
print(len(ids), len(bibnames),len(df_main))

0 5693 5693


In [ ]:
# df_main['IDs'] = ids
df_main['bib name'] = bibnames
df_main.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv',index=False)

###### Use one dataset containing bname, bibname of bname, id of bname and add derivative works to it using task 1 poolA_derworks etc



In [ ]:
df_poolA_bd = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolA_derworks.csv')
df_poolB_bd = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolB_derworks.csv')
df_poolC_bd = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative Works (Task 1)/poolC_derworks.csv')

df_poolA_bd.head()

,bName,dNames,dIDs
0,Helicopter trimming and tracking control using...,Adaptive Dynamic Programming: An Introduction ...,2109b7c5067edd88f031cd33ed4d2655f627b7f1 #;# ...
1,The Infinite Gaussian Mixture Model,AMS 241: Bayesian Nonparametric Methods (Fall ...,7c48b8b6b2805f13e5a38a293c76203f1a3ac629 #;# ...
2,Derandomizing stochastic prediction strategies,"Prediction, learning, and games #;# Competit...",9fa9777415729cebfcbcb5ee667057e981f9912e #;# ...
3,Approximate Planning in Large POMDPs via Reusa...,Representation of a State Value Function No Ye...,6dd273e335fdc1a19e817dbf1eabd1bf743c9fa4 #;# ...
4,Ensemble Adversarial Training Attacks and Defe...,Adversarial Examples in Modern Machine Learnin...,7f1e602a44b56b9853fcc2063df9593e7b79ba22 #;# ...


In [ ]:
df_main.head()

,title,authors,n_citation,doi,url,pdf,conf,fos,year,pool,topic,labels,bib name
0,Revisiting Frank-Wolfe: Projection-Free Sparse...,Martin Jaggi,502,NaN,NaN,NaN,International Conference on Machine Learning,Mathematical optimization=0.45817 ; Computer s...,2013,A,None,mo,revisiting frankwolfe projectionfree sparse co...
1,Interactive storytelling: a player modelling a...,David ThueVadim Bulitko ; Marcia Spetch ; Eric...,134,NaN,NaN,NaN,National Conference on Artificial Intelligence,Entertainment=0.49793 ; Computer science=0.440...,2007,C,None,ml,interactive storytelling a player modelling ap...
2,Knowledge compilation using horn approximations,Bart SelmanHenry Kautz ;,143,NaN,NaN,NaN,National Conference on Artificial Intelligence,Knowledge representation and reasoning=0.55433...,1991,C,AI,dl,NaN
3,Solving large-scale constraint satisfaction an...,Steven MintonMark D. Johnston ; Andrew B. Phil...,269,NaN,NaN,NaN,National Conference on Artificial Intelligence,Constraint satisfaction=0.62121 ; Mathematical...,1990,C,None,mo,solving largescale constraintsatisfaction and ...
4,Regularization of Neural Networks using DropCo...,Li WanMatthew Zeiler ; Sixin Zhang ; Yann Le C...,1120,NaN,NaN,NaN,International Conference on Machine Learning,Pattern recognition=0.45948 ; Computer science...,2013,A,ML,ml,regularization of neural networks using dropco...


In [ ]:
dict_bd = dict()

for index,row in df_poolA_bd.iterrows():
  dict_bd[clean(row['bName'])] = row['dIDs']

for index,row in df_poolB_bd.iterrows():
  dict_bd[clean(row['bName'])] = row['dIDs']

for index,row in df_poolC_bd.iterrows():
  dict_bd[clean(row['bName'])] = row['dIDs']

In [ ]:
len(dict_bd)

5970

In [ ]:
l = list(dict_bd.keys())
l.count('')

0

In [ ]:
dids = []

for index,row in df_main.iterrows():
  if(isinstance(row['bib name'],str)):
      dids.append(dict_bd[row['bib name']])
  else:
    dids.append('')

In [ ]:
len(dids)

5693

In [ ]:
df_main['dIDs'] = dids
df_main.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv',index=False)
df_main.head()

,title,authors,n_citation,doi,url,pdf,conf,fos,year,pool,topic,labels,bib name,dIDs
0,Revisiting Frank-Wolfe: Projection-Free Sparse...,Martin Jaggi,502,NaN,NaN,NaN,International Conference on Machine Learning,Mathematical optimization=0.45817 ; Computer s...,2013,A,None,mo,revisiting frankwolfe projectionfree sparse co...,aa59b3facccb4d40377d5a08ce0d2d26fec8c200 #;# ...
1,Interactive storytelling: a player modelling a...,David ThueVadim Bulitko ; Marcia Spetch ; Eric...,134,NaN,NaN,NaN,National Conference on Artificial Intelligence,Entertainment=0.49793 ; Computer science=0.440...,2007,C,None,ml,interactive storytelling a player modelling ap...,f8b4151335847dae10fa6266cad3ec4f6595ba6d #;# ...
2,Knowledge compilation using horn approximations,Bart SelmanHenry Kautz ;,143,NaN,NaN,NaN,National Conference on Artificial Intelligence,Knowledge representation and reasoning=0.55433...,1991,C,AI,dl,NaN,
3,Solving large-scale constraint satisfaction an...,Steven MintonMark D. Johnston ; Andrew B. Phil...,269,NaN,NaN,NaN,National Conference on Artificial Intelligence,Constraint satisfaction=0.62121 ; Mathematical...,1990,C,None,mo,solving largescale constraintsatisfaction and ...,341e501790209f178ded2bd6b8729add7ec8bb02 #;# ...
4,Regularization of Neural Networks using DropCo...,Li WanMatthew Zeiler ; Sixin Zhang ; Yann Le C...,1120,NaN,NaN,NaN,International Conference on Machine Learning,Pattern recognition=0.45948 ; Computer science...,2013,A,ML,ml,regularization of neural networks using dropco...,c1f05b723e53ac4eb1133249b445c0011d42ca79 #;# ...


## Now to one_data_v2, add ddIDs
<p> to do this, go to mapped_dName_dID_PoolA_abovemed.csv and get dNameIDs and ddIDs --> make dict</p>
<p>using dnameIDs in one_data_v2.dIds, make list of lists separated by \space\$;$\space\</p>

In [ ]:
col_names1 = ['ddIDs',	'dNameIDs']
col_names = ['ddIDs',	'mapped_dNameIDs']

df_a_ab = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_dName_dID_PoolA_abovemed.csv',usecols=col_names1)
df_a_be = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_dName_dID_PoolA_belowmed.csv',usecols=col_names)

df_b_ab = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_B_abovemed.csv',usecols=col_names)
df_b_be = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_B_belowmed.csv',usecols=col_names)

df_c_ab = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_C_abovemed.csv',usecols=col_names)
df_c_be = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Derivative_works_of_derivative_works_Task1.5/mapped_C_belowmed.csv',usecols=col_names)

In [ ]:
def clean_liststr(s):
  if(not isinstance(s,str)):
    return('')
  if('[' in s):
    s = s.strip()
    s = s.split('[')[1]
    s = s.split(']')[0]
    s = s.split(',')
    s = [x.strip() for x in s]

    s = ' #$# '.join(s)
    return(s)
  else:
    # print(s)
    return(s)

In [ ]:
dID_ddID_map = dict()

def add_to_dict(d,df,did='mapped_dNameIDs'):
  for index,row in df.iterrows():
    if(isinstance(row[did],str)):
      d[row[did].strip()] = clean_liststr(row['ddIDs'])
    else:
      if(not isinstance(row[did],float)):
        print(type(row[did]))

  return(d)

In [ ]:
dID_ddID_map = add_to_dict(dID_ddID_map,df_a_ab,'dNameIDs')
dID_ddID_map = add_to_dict(dID_ddID_map,df_a_be)

dID_ddID_map = add_to_dict(dID_ddID_map,df_b_ab)
dID_ddID_map = add_to_dict(dID_ddID_map,df_b_be)

dID_ddID_map = add_to_dict(dID_ddID_map,df_c_ab)
dID_ddID_map = add_to_dict(dID_ddID_map,df_c_be)

In [ ]:
len(dID_ddID_map)

14790

In [ ]:
list(dID_ddID_map.keys())[0]

'e774e704898d6292a03ee38e022872c52c6bf272'

In [ ]:
ddIDs = []
df = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv')

dIDs = list(df['dIDs'])
count_dids = 0
count_err = 0
i=0

for dID in dIDs:
  if(isinstance(dID,str)):
    ids = dID.split(' #;# ')
    ids = [x.strip() for x in ids]
    # print('d:',len(ids),ids)
    count_dids+=len(ids)
    dd = []

    for id in ids:
      try:
        dd.append(dID_ddID_map[id])
      except:
        dd.append('')
        count_err+=1
        # print(id)
    
    if(len(dd)!=len(ids)):
      print(i)
      break

    i+=1
    dd = ' #;# '.join(dd)
    ddIDs.append(dd)

    dd = dd.split(' #;# ')
    dd = [x.split(' #$# ') for x in dd]
    # print(len(dd),len(ids))

    # print(ddIDs)

    x = ddIDs[0]
    x = x.split(' #;# ')
    # print(len(x))
    # break

  else:
    ddIDs.append('')

In [ ]:
count_err

15421

In [ ]:
count_dids

36931

In [ ]:
len(ddIDs)

5693

In [ ]:
df['ddIDs'] = ddIDs
df.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv')

In [ ]:
print(df['ddIDs'])

0       'aa59b3facccb4d40377d5a08ce0d2d26fec8c200' #$#...
1            #;#  #;#  #;#  #;#  #;#  #;#  #;#  #;#  #;# 
2                                                     NaN
3            #;#  #;#  #;#  #;#  #;#  #;#  #;#  #;#  #;# 
4       '8452941a175c899628c679523da952b18f63e335' #$#...
                              ...                        
5688                                                  NaN
5689                                                  NaN
5690    '78e173dae2c7d9ae0b525a60c1443443c5c7342e' #$#...
5691                                                  NaN
5692                                                  NaN
Name: ddIDs, Length: 5693, dtype: object


In [ ]:
for d in ddIDs:
  print(d)

'aa59b3facccb4d40377d5a08ce0d2d26fec8c200' #$# '7c2543b2c84613c2397ebf61d55797663501f72f' #$# '26c00887f5b85ffd69377f5de66e01be950fab49' #$# '9ef93ef58b2bb8a5045753ba888fc3e4e10b004d' #$# '2c51e647dc2f2bcbab3f989feba274977cb71114' #$# '5c6cafd7979ea486468208ac424a4ef046ff449f' #$# 'f81dd1680b2b6101256a5de6ac1f5df3d5604559' #$# 'fb8d7f75e00c38aa79335184b746ee884d0a72d8' #$# 'e0d50067af445aa32fe1fadc4a4ce9297e4911c9' #$# '5884b618761519add502e692c007a871a4a20f95' #;# '96ba005687c3bebdd605be8606017d9b44f42f13' #$# '3c70c7407e2694d7ebe4fc88cd13829704c8f848' #$# '8fcd5b558abd0f856fdb4cdc894f583c29166db3' #$# '730a870cd046240ee1e635af914b3ada6d91c882' #$# 'afba27096a777d97410959ed18defcd273b9f6be' #$# '48e63bd7d8268b432606c8f15128d0740e2e55dc' #$# '333be18b5f9e379da096e33aff47cede6b071956' #$# 'a1d79d962f0bdc536fe911ad0a057ebb4d3da3d6' #$# '3248949f4f7d3b7753c3f8e6c57216658baed8b6' #$# '9ef93ef58b2bb8a5045753ba888fc3e4e10b004d' #;# '730a870cd046240ee1e635af914b3ada6d91c882' #$# 'afba27096a77

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Remove entries with no dID and ddIDs

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv')
df = df.drop(['Unnamed: 0','Unnamed: 0.1','doi','url','pdf'],axis=1)
df.columns

Index(['title', 'authors', 'n_citation', 'conf', 'fos', 'year', 'pool',
       'topic', 'labels', 'bib name', 'dIDs', 'ddIDs'],
      dtype='object')

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset.csv')
ids = df2['IDs']
len(ids)

5693

In [ ]:
len(df)

5693

In [ ]:
df['bIDs'] = ids

In [ ]:
df = df.dropna()
len(df)

3697

In [ ]:
df['bIDs'] = list(range(len(df)))

In [ ]:
df.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_final.csv',index=False)